In [26]:
import pandas as pd
import matplotlib.pyplot as plt
from client_analiser.models import ModelInterface, ModelA, ModelB

In [13]:
iteration_path = "iteration_3/"
deliveries_path = "../data/" + iteration_path + "raw/deliveries.jsonl"
products_path = "../data/" + iteration_path + "raw/products.jsonl"
sessions_path = "../data/" + iteration_path + "raw/sessions.jsonl"
users_path = "../data/" + iteration_path + "raw/users.jsonl"

In [14]:
deliveries_data = pd.read_json(deliveries_path, lines=True)
products_data = pd.read_json(products_path, lines=True)
sessions_data = pd.read_json(sessions_path, lines=True)
users_data = pd.read_json(users_path, lines=True)

In [17]:
sessions_data = sessions_data.sort_values(by=['timestamp'])
# sessions_data['timestamp_date'] = sessions_data['timestamp'].apply(lambda x: x.date())
sessions_data['timestamp_week'] = sessions_data['timestamp'].apply(lambda x: x.week)
sessions_data['timestamp_month'] = sessions_data['timestamp'].apply(lambda x: x.month)
sessions_data['timestamp_quarter'] = sessions_data['timestamp'].apply(lambda x: x.quarter)

In [18]:
train_data = sessions_data[sessions_data.timestamp_month < 12]
test_data = sessions_data[sessions_data.timestamp_month >= 12]

In [27]:
# Models
model_A: ModelInterface = ModelA()
model_B: ModelInterface = ModelB()

In [ ]:
model_B

In [21]:
predictionsA = model_A.predict_expenses(products_data, deliveries_data, train_data, users_data)
predictionsA

C:\Users\micha\Desktop\Informatyka\Inżynieria_uczenia_maszynowego\Projekt\IUM-z5w1\client_analiser\models\model_a.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions["timestamp"] = pd.to_datetime(sessions["timestamp"])
C:\Users\micha\Desktop\Informatyka\Inżynieria_uczenia_maszynowego\Projekt\IUM-z5w1\client_analiser\models\model_a.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions['timestamp_interval'] = sessions['timestamp'].apply(lambda x: x.month)


,user_expenses
user_id,
139,180.71
242,533.24
108,1718.53
143,830.67
140,3541.68
...,...
289,0.00
225,33.69
152,0.00


In [22]:
def get_user_id_from_session(session):
    sample_user_id = session['user_id'].iloc[0]
    for user_id in session['user_id']:
        if sample_user_id != user_id:
            raise Exception("How it is even possible")
    return sample_user_id

def get_user_expenses(user_session_data):
    d = {
        'user_id': get_user_id_from_session(user_session_data),
        'expenses': user_session_data[user_session_data['event_type'] == "BUY_PRODUCT"]['price'].sum()
    }
    # df = pd.DataFrame(data=d)
    return d


def calculate_expenses(sessions_data, products_data, users_data):
    enriched_sessions_data = pd.merge(sessions_data, products_data, on="product_id").sort_values(by=['timestamp'])
    user_expenses = []
    for user_id in range(users_data['user_id'].min(), users_data['user_id'].max() + 1):
        # for user_id in enriched_sessions_data['user_id'].unique():
        user_session_data = enriched_sessions_data[enriched_sessions_data['user_id'] == user_id]
        user_expenses.append(
            {
                'user_id': user_id,
                'expenses': user_session_data[user_session_data['event_type'] == "BUY_PRODUCT"]['price'].sum()
            }
        )
        # user_expenses.append(get_user_expenses(enriched_sessions_data[enriched_sessions_data['user_id'] == user_id]))
    return pd.DataFrame(data=user_expenses).set_index('user_id')


def loss(predictions, observations):
    unified_data = pd.merge(predictions, observations, on="user_id").sort_values(by=['user_id'])
    unified_data['difference'] = unified_data['user_expenses'] - unified_data['expenses']
    unified_data['difference_square'] = unified_data['difference'].apply(lambda x: x ** 2)
    return unified_data

In [23]:
observations = calculate_expenses(test_data, products_data, users_data)

In [25]:
loss(predictionsA, observations)

,user_expenses,expenses,difference,difference_square
user_id,,,,
102,161.00,0.00,161.00,2.592100e+04
103,4688.73,554.77,4133.96,1.708963e+07
104,4476.97,2332.01,2144.96,4.600853e+06
105,1035.18,0.00,1035.18,1.071598e+06
106,3728.79,0.00,3728.79,1.390387e+07
...,...,...,...,...
297,2054.55,109.00,1945.55,3.785165e+06
298,3417.61,2399.00,1018.61,1.037566e+06
299,3040.79,0.00,3040.79,9.246404e+06
